In [3]:
from imp import reload
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib as mpl
import nltk,re,pprint
import sys,glob,os
import operator, string, argparse, math

In [9]:
class flingDBSCAN:
    def __init__(self,data,nthres):
        self.data = data
        self.near_threshold = nthres
        self.nDocs = len(self.data)
        print("\nDBSCAN initialized!\n")
                
    def distanceBtnTwoDocs(self, docId_1, docId_2):
        listWords_1 = set(list(self.data['tfMatrix'][docId_1]['word']))
        listWords_2 = set(list(self.data['tfMatrix'][docId_2]['word']))
        common = listWords_1.intersection(listWords_2)
        diff1_2 = listWords_1.difference(listWords_2)
        diff2_1 = listWords_2.difference(listWords_1)
        sumwt1 = self.data['sumTFIDF'][docId_1]
        sumwt2 = self.data['sumTFIDF'][docId_2]
        score_common, score_doc1, score_doc2 = 0,0,0
        for word_c in common:
            score_1 = float(self.data['tfMatrix'][docId_1].loc[self.data['tfMatrix'][docId_1]['word'] == word_c]['tf-idf'])
            score_2 = float(self.data['tfMatrix'][docId_2].loc[self.data['tfMatrix'][docId_2]['word'] == word_c]['tf-idf'])
            score_common += abs(score_1/float(sumwt1) - score_2/float(sumwt2))
        for word_d12 in diff1_2:
            score_1 = float(self.data['tfMatrix'][docId_1].loc[self.data['tfMatrix'][docId_1]['word'] == word_d12]['tf-idf'])
            score_doc1 += score_1/float(sumwt1)
        for word_d21 in diff2_1:
            score_2 = float(self.data['tfMatrix'][docId_2].loc[self.data['tfMatrix'][docId_2]['word'] == word_d21]['tf-idf'])
            score_doc2 += score_2/float(sumwt2)
        score_total = score_common + score_doc1 + score_doc2
        return(score_total)
    
    def findNear(self,docId):
        nearList = {}
        for el in range(self.nDocs):
            if docId!=el:
                dist = self.distanceBtnTwoDocs(int(docId),int(el))
                if dist>=self.near_threshold:
                    nearlist[[docId,el]]=dist
        return (nearlist)

In [4]:
dataProcessed = pd.read_csv('../processFiles/data_tfidf_processed.csv', encoding="latin")

In [10]:
fdb = flingDBSCAN(dataProcessed,1.75)
print(fdb.findNear(1))


DBSCAN initialized!



TypeError: string indices must be integers